In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('Musical_instruments_reviews.csv')
df = df.dropna()
display(df.head())

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A2IBPI20UZIR0U,1384719342,"cassandra tu ""Yeah, well, that's just like, u...","[0, 0]","Not much to write about here, but it does exac...",5.0,good,1393545600,"02 28, 2014"
1,A14VAT5EAX3D9S,1384719342,Jake,"[13, 14]",The product does exactly as it should and is q...,5.0,Jake,1363392000,"03 16, 2013"
2,A195EZSQDW3E21,1384719342,"Rick Bennette ""Rick Bennette""","[1, 1]",The primary job of this device is to block the...,5.0,It Does The Job Well,1377648000,"08 28, 2013"
3,A2C00NNG1ZQQG2,1384719342,"RustyBill ""Sunday Rocker""","[0, 0]",Nice windscreen protects my MXL mic and preven...,5.0,GOOD WINDSCREEN FOR THE MONEY,1392336000,"02 14, 2014"
4,A94QU4C90B1AX,1384719342,SEAN MASLANKA,"[0, 0]",This pop filter is great. It looks and perform...,5.0,No more pops when I record my vocals.,1392940800,"02 21, 2014"


In [2]:
def convert_to_lower(column):
    return column.str.lower()

df['reviewText'] = df['reviewText'].str.lower()
df['reviewerName'] = df['reviewerName'].str.lower()
display(df.head())

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A2IBPI20UZIR0U,1384719342,"cassandra tu ""yeah, well, that's just like, u...","[0, 0]","not much to write about here, but it does exac...",5.0,good,1393545600,"02 28, 2014"
1,A14VAT5EAX3D9S,1384719342,jake,"[13, 14]",the product does exactly as it should and is q...,5.0,Jake,1363392000,"03 16, 2013"
2,A195EZSQDW3E21,1384719342,"rick bennette ""rick bennette""","[1, 1]",the primary job of this device is to block the...,5.0,It Does The Job Well,1377648000,"08 28, 2013"
3,A2C00NNG1ZQQG2,1384719342,"rustybill ""sunday rocker""","[0, 0]",nice windscreen protects my mxl mic and preven...,5.0,GOOD WINDSCREEN FOR THE MONEY,1392336000,"02 14, 2014"
4,A94QU4C90B1AX,1384719342,sean maslanka,"[0, 0]",this pop filter is great. it looks and perform...,5.0,No more pops when I record my vocals.,1392940800,"02 21, 2014"


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10227 entries, 0 to 10260
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   reviewerID      10227 non-null  object 
 1   asin            10227 non-null  object 
 2   reviewerName    10227 non-null  object 
 3   helpful         10227 non-null  object 
 4   reviewText      10227 non-null  object 
 5   overall         10227 non-null  float64
 6   summary         10227 non-null  object 
 7   unixReviewTime  10227 non-null  int64  
 8   reviewTime      10227 non-null  object 
dtypes: float64(1), int64(1), object(7)
memory usage: 799.0+ KB


In [4]:
df = df.fillna(value='')

import re
def remove_special_chars(text):
    if isinstance(text, str):  # Verifica se é uma string
        cleaned_text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
        return cleaned_text
    return text

df['reviewText'] = df['reviewText'].apply(remove_special_chars)
df['reviewerName'] = df['reviewerName'].apply(remove_special_chars)

In [5]:
import nltk 
from nltk.tokenize import word_tokenize

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

def preprocess_text(text):
    #Tokenização
    tokens = word_tokenize(text)
    clean_tokens = [token.lower() for token in tokens if re.match(r'^[a-zA-Z]+$', token)]
    filtered_tokens = [token for token in clean_tokens if token not in stopwords.words('english')]
    
    clean_text = ' '.join(filtered_tokens)
    return clean_text

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\pedri\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\pedri\AppData\Roaming\nltk_data...
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pedri\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [19]:
#Tranformando coluna de avaliações em binário 1 para bom e 0 para ruim
df['overall'] = df['overall'].apply(lambda x: 1 if x > 3 else 0)
print(df.overall.value_counts())


#Diminuir o tamanho do dataset
count_class_0 = (df['overall'] == 0).sum()
count_class_1 = (df['overall'] == 1).sum()
if count_class_0 >= 1200 and count_class_1 >= 1200:
    q1 = df[df.overall == 0].sample(1200, replace=True)
    q2 = df[df.overall == 1].sample(1200, replace=True)

    df = pd.concat([q1, q2], axis=0)
    print(df.overall.value_counts())
    print(df.columns)
    print(df.shape)

0    10227
Name: overall, dtype: int64


In [13]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(df.reviewText, df.overall, test_size=0.2, random_state=42)
#BOW
from sklearn.feature_extraction.text import CountVectorizer
v = CountVectorizer()
print(X_train)
X_train_bow = v.fit_transform(X_train.values)
X_train_bow.toarray()[:3]

4564    black tortex picks with silver letteringthe ja...
932     i recently ordered a wide variety of picks to ...
8457    i love these cheap little pedalsi dont know if...
7650    this real leather strap is nice the price is e...
8955    i have never played a string instrument and it...
                              ...                        
5754    the cable is good and normal  good price and w...
5210    unlike those who have complained that they dry...
5409    what an amazing product multivoltage capabilit...
865     as a vast multitude of guitar players will sol...
7292    a modeling amp will literally model existing a...
Name: reviewText, Length: 8181, dtype: object


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [14]:
#Utilizando o método de Naive Bytes
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()
model.fit(X_train_bow, y_train)

MultinomialNB()

In [17]:
#Testando o modelo
X_test_bow = v.transform(X_test)
model.score(X_test_bow, y_test)

1.0